In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.statespace.sarimax import SARIMAX
import yfinance
import warnings
warnings.filterwarnings('ignore')


data = pd.read_csv('data\df.csv',sep='|')
data['date'] = pd.to_datetime(data['date'])

#plt.figure(figsize=(12, 6))
#plt.plot(ts)
#plt.title('Prix journaliers du S&P500')
#plt.show()

DXY = yfinance.download('DX-Y.NYB', start='1999-01-04', end='2025-09-19')
DXY = DXY.reset_index()[['Date','Close']]
DXY.columns = ['date','DXY']
data=data.merge(DXY, on='date', how='left')

GDPM = pd.read_csv('GDP Monthly.csv',sep=',')
GDPM['date'] = pd.to_datetime(GDPM['timestamp'])
GDPM['year'] = GDPM['date'].dt.year
GDPM['month'] = GDPM['date'].dt.month
GDPM['CroissM'] = GDPM['EIA/GDPQXUS/USA'].diff()

data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

data = data.merge(GDPM, on=['year', 'month'], how='left')
data.drop(['timestamp','date_y','year','month'], axis=1, inplace=True)
data['DXY'].fillna(method='ffill', inplace=True)
data.set_index('date_x', inplace=True)

[*********************100%***********************]  1 of 1 completed


SARIMAX : (sans le code qui optimise les données)
Prix cible : 6693.
(1,1,1) = 62471.633 (6691.68)
(2,1,1) = 62569.103 (6695.99)
(3,1,1) = 62434.749 (6690.88)
(4,1,1) = 62424.398 (6690.97)

(1,1,2) = 62432.014 (6690.70)
(1,1,3) = 62441.745 (6689.84)
(1,1,4) = 63080.847 (6679.74)

(2,1,2) = 67140.571 (6679.01)
(3,1,2) = 73526.320 (6653.08)
(4,1,2) = 73828.371 (6650.60)

(2,1,3) = 63628.154 (6680.44)
(2,1,4) = 63308.054 (6680.21)

(3,1,2) = 73526.320 (6653.08)
(3,1,3) = 63474.356 (6679.84)
(3,1,4) = 63801.005 (6682.05)

(4,1,3) = 65747.309 (6681.32)
(4,1,4) = 62756.699 (6682.55)

SARIMAX : (avec le code qui optimise les données)
Prix cible : 6691.
(1,1,1) = 62435.538 (6690.87)
(2,1,1) = 62432.247 (6689.72)
(3,1,1) = 62430.131 (6690.50)
(4,1,1) = 62418.825 (6690.62)

(1,1,2) = 62415.196 (6690.17)
(1,1,3) = 62400.380 (6689.43)
(1,1,4) = 62403.129 (6686.57)


SARIMAX : 
(1,0,1,5) = 62435.538 (6690.87)
(2,0,1,5) = 62393.235 (6690.45)
(3,0,1,5) = 62356.608 (6691.98)
(4,0,1,5) = 62298.040 (6691.60)

(1,0,2,5)= 62386.291 (6690.43)
(1,0,3,5)= 62331.620 (6691.78)
(1,0,4,5)= 62288.145 (6691.59)

(1,1,1)*(3,0,1,5) = 62356.608 (6691.98)
(2,1,1)* = 62331.584 (6690.63)
(3,1,1)* = 62318.968 (6689.36)
(4,1,1)* = 

(2,1,2)* = 62340.088 (6691.80)
(3,1,2)* = 62442.106 (6688.75)

SARIMAX Utilisé : 
(1,1,1)*(3,0,1,5)

In [123]:
class SP500SARIMAXForecaster:
    """
    Modèle SARIMAX pour S&P 500 avec variables exogènes
    """
    
    def __init__(self):
        self.model = None
        self.fitted_model = None
        self.data = None
        self.exog_vars = None
        self.feature_names = []
        self.is_fitted = False
        
    def load_data(self, file_path):
        """
        Charge les données depuis le fichier Excel
        """
        try:
            # Lire le fichier Excel
            df = data
            
            # Détecter la colonne de date
            date_cols = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
            if date_cols:
                df[date_cols[0]] = pd.to_datetime(df[date_cols[0]])
                df.set_index(date_cols[0], inplace=True)
            else:
                print("Aucune colonne de date détectée. Veuillez spécifier manuellement.")
                
            self.data = df
            return df
            
        except Exception as e:
            print(f"Erreur lors du chargement: {e}")
            return None
    
    def create_market_features(self, target_column='cible'):
        """
        Crée des variables techniques et calendaires utiles pour le S&P 500
        
        Parameters:
        -----------
        target_column : str
            Nom de la colonne contenant les prix du S&P 500
        """
        if self.data is None:
            raise ValueError("Données non chargées. Utilisez load_data() d'abord.")
            
        df = self.data.copy()
        
        # === VARIABLES TECHNIQUES ===
        
        # 1. Rendements et volatilité
        if target_column in df.columns:
            df['returns'] = df[target_column].pct_change()
            df['log_returns'] = np.log(df[target_column] / df[target_column].shift(1))
            
            # Volatilité réalisée (rolling)
            df['volatility_5d'] = df['returns'].rolling(5).std()
            df['volatility_20d'] = df['returns'].rolling(20).std()
        
        # 2. Moyennes mobiles et signaux techniques
        # Signaux techniques
        df['ma_signal_short'] = (df[target_column] > df['mov_avg_10d']).astype(int)
        df['ma_signal_long'] = (df[target_column] > df['mov_avg_30d']).astype(int)
        
        # 3. Volume (si disponible)
        if 'volume' in df.columns:
            df['volume_ma_20'] = df['volume'].rolling(20).mean()
            df['volume_ratio'] = df['volume'] / df['volume_ma_20']
        
        # === VARIABLES MACRO-ÉCONOMIQUES (si disponibles) ===        
        
        self.data = df
        return df
    
    def select_exog_variables(self, target_column='cible', method='all'):
        """
        Sélectionne les meilleures variables exogènes
        
        Parameters:
        -----------
        target_column : str
            Variable cible (S&P 500)
        method : str
            'correlation', 'all', ou liste des noms de variables
        """
        if self.data is None:
            raise ValueError("Données non disponibles.")
            
        # Variables candidates (exclure la cible et variables avec trop de NaN)
        exclude_cols = [target_column, 'returns', 'log_returns'] + \
                      [col for col in self.data.columns if self.data[col].isna().sum() > len(self.data) * 0.1]
        
        candidate_vars = [col for col in self.data.columns if col not in exclude_cols and 
                         self.data[col].dtype in ['int64', 'float64', 'bool']]
        
        if method == 'all':
            selected_vars = candidate_vars
        elif method == 'correlation':
            # Sélection basée sur la corrélation avec les rendements
            if 'returns' in self.data.columns:
                correlations = self.data[candidate_vars + ['returns']].corr()['returns'].abs().sort_values(ascending=False)
                # Prendre les 15 variables les plus corrélées
                selected_vars = correlations.head(16).index.tolist()
                selected_vars.remove('returns') if 'returns' in selected_vars else None
            else:
                selected_vars = candidate_vars[:15]  # Limiter à 15 variables
        elif isinstance(method, list):
            selected_vars = [var for var in method if var in candidate_vars]
        else:
            selected_vars = candidate_vars[:10]
        
        # Préparer les données exogènes
        exog_data = self.data[selected_vars].fillna(method='ffill').fillna(0)
        self.exog_vars = exog_data
        self.feature_names = selected_vars
        
        for i, var in enumerate(selected_vars, 1):
            print(f"{i:2d}. {var}")
            
        # Montrer les corrélations si disponibles
        if 'returns' in self.data.columns:
            print(f"\nCorrélations avec les rendements:")
            corr_with_returns = self.data[selected_vars + ['returns']].corr()['returns'].abs().sort_values(ascending=False)
            for var in corr_with_returns.head(5).index:
                if var != 'returns':
                    print(f"   {var}: {corr_with_returns[var]:.3f}")
        
        return exog_data
    
    def fit_sarimax_model(self, target_column='close', 
                         order=(1,1,1), seasonal_order=(3,0,1,5)):
        """
        Ajuste le modèle SARIMAX avec variables exogènes
        """
        if self.data is None or self.exog_vars is None:
            raise ValueError("Données ou variables exogènes non préparées.")
            
        target_series = self.data[target_column].dropna()
        
        # Aligner les données exogènes avec la série cible
        common_index = target_series.index.intersection(self.exog_vars.index)
        target_aligned = target_series.loc[common_index]
        exog_aligned = self.exog_vars.loc[common_index]
        
        print(f"Ajustement SARIMAX{order}x{seasonal_order}")
        print(f"Données: {len(target_aligned)} observations")
        print(f"Variables exogènes: {len(self.feature_names)}")
        
        try:
            self.model = SARIMAX(
                target_aligned,
                exog=exog_aligned,
                order=order,
                seasonal_order=seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False,
                trend='t'
            )
            
            self.fitted_model = self.model.fit(disp=False)
            self.is_fitted = True
            
            print(f"Modèle ajusté avec succès!")
            print(f"AIC: {self.fitted_model.aic:.2f}")
            print(f"Log-likelihood: {self.fitted_model.llf:.2f}")
            
            return self.fitted_model
            
        except Exception as e:
            print(f"Erreur lors de l'ajustement: {e}")
            return None
    
    def forecast_with_exog(self, n_periods=10, exog_forecast=None):
        """
        Génère des prévisions avec variables exogènes
        
        Parameters:
        -----------
        n_periods : int
            Nombre de périodes à prédire
        exog_forecast : DataFrame, optional
            Valeurs futures des variables exogènes
            Si None, utilise la dernière valeur observée
        """
        if not self.is_fitted:
            raise ValueError("Modèle non ajusté.")
            
        if exog_forecast is None:
            # Utiliser les dernières valeurs pour les variables exogènes
            last_exog = self.exog_vars.iloc[-1:].copy()
            exog_forecast = pd.concat([last_exog] * n_periods, ignore_index=True)
        
        try:
            forecast = self.fitted_model.forecast(steps=n_periods, exog=exog_forecast)
            conf_int = self.fitted_model.get_forecast(steps=n_periods, exog=exog_forecast).conf_int()
            
            return forecast, conf_int
            
        except Exception as e:
            print(f"Erreur lors de la prévision: {e}")
            return None, None
    
    def plot_forecast(self, target_column='cible', n_periods=10, figsize=(15, 8)):
        """
        Visualise les prévisions SARIMAX
        """
        if not self.is_fitted:
            raise ValueError("Modèle non ajusté.")
            
        forecast, conf_int = self.forecast_with_exog(n_periods)
        
        if forecast is None:
            return
            
        # Données récentes
        recent_data = self.data[target_column].dropna()[-60:]
        
        # Dates futures
        last_date = recent_data.index[-1]
        future_dates = pd.bdate_range(start=last_date + pd.Timedelta(days=1), periods=n_periods)
        
        plt.figure(figsize=figsize)
        
        # Données historiques
        plt.plot(recent_data.index, recent_data, 
                label='S&P 500 Historique', color='black', linewidth=2)
        
        # Prévisions
        plt.plot(future_dates, forecast, 
                label=f'Prévisions SARIMAX ({n_periods}j)', 
                color='red', linewidth=2)
        
        # Intervalles de confiance
        plt.fill_between(future_dates, conf_int.iloc[:, 0], conf_int.iloc[:, 1],
                        color='red', alpha=0.2, label='Intervalle de confiance 95%')
        
        plt.title('S&P 500 - Prévisions SARIMAX avec Variables Exogènes', fontsize=16)
        plt.xlabel('Date', fontsize=12)
        plt.ylabel('Valeur de l\'indice', fontsize=12)
        plt.legend(fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        return forecast, conf_int
    
    def feature_importance(self):
        """
        Analyse l'importance des variables exogènes
        """
        if not self.is_fitted:
            raise ValueError("Modèle non ajusté.")
            
        # Coefficients des variables exogènes
        params = self.fitted_model.params
        # Prendre tous les coefficients qui sont dans feature_names
        exog_params = params[params.index.isin(self.feature_names)]
        
        if len(exog_params) > 0:
            # Créer un DataFrame avec les coefficients
            importance_df = pd.DataFrame({
                'Variable': self.feature_names[:len(exog_params)],
                'Coefficient': exog_params.values,
                'Abs_Coefficient': np.abs(exog_params.values),
                'P_Value': self.fitted_model.pvalues[exog_params.index].values
            }).sort_values('Abs_Coefficient', ascending=False)
            
            print("=== IMPORTANCE DES VARIABLES EXOGÈNES ===")
            print(importance_df.head(10).to_string(index=False))
            
            return importance_df
        else:
            print("Aucune variable exogène détectée dans le modèle.")
            return None
    
    def summary(self):
        """Résumé du modèle"""
        if self.fitted_model:
            return self.fitted_model.summary()
        else:
            print("Modèle non ajusté.")


# Initialiser le forecaster
forecaster = SP500SARIMAXForecaster()
forecaster.data = data

# 1. Créer les variables techniques
print("1. Création des variables techniques...")
forecaster.create_market_features(target_column='cible')  # Remplacez 'cible' par votre colonne prix

# 2. Sélectionner les variables exogènes
exog_vars = forecaster.select_exog_variables(target_column='cible', method='all')

# 3. Ajuster le modèle SARIMAX
fitted_model = forecaster.fit_sarimax_model(
    target_column='cible',
    order=(1,1,1),
    seasonal_order=(3,0,1,5)
)

if fitted_model:
    # 4. Analyser l'importance des variables
    importance = forecaster.feature_importance()
    
    # 4bis. AJOUTER ICI LA MÉTHODE 1 (optimisation)
    print("\n4bis. OPTIMISATION DES VARIABLES")
    print("=" * 60)
    
    # Filtrer les variables significatives (p-value < 0.05)
    significant_vars = importance[importance['P_Value'] < 0.05]['Variable'].tolist()
    
    print(f"✓ Variables significatives (p < 0.05): {len(significant_vars)}/{len(importance)}")
    print(f"Variables retenues: {significant_vars}")
    
    # Réajuster le modèle avec seulement les variables significatives
    print("\nRé-ajustement avec variables significatives...")
    forecaster.select_exog_variables(target_column='cible', method=significant_vars)
    
    fitted_model_opt = forecaster.fit_sarimax_model(
        target_column='cible',
        order=(1,1,1),
        seasonal_order=(3,0,1,5)
    )
    
    # Comparer les AIC
    print(f"\nComparaison:")
    print(f"  AIC modèle initial:    {fitted_model.aic:.2f}")
    print(f"  AIC modèle optimisé:   {fitted_model_opt.aic:.2f}")
    print(f"  Amélioration:          {fitted_model.aic - fitted_model_opt.aic:.2f}")
    
    
    # 5. Générer des prévisions
    forecast, conf_int = forecaster.plot_forecast(target_column='cible', n_periods=10)
    
    # Extraire les 30 dernières valeurs observées
    last_30 = forecaster.data['cible'].dropna()[-20:]

    # Créer les dates futures (si pas déjà fait)
    last_date = last_30.index[-1]
    future_dates = pd.bdate_range(start=last_date + pd.Timedelta(days=1), periods=len(forecast))

    # Concaténer les valeurs et les dates pour un seul plot si besoin
    all_values = pd.concat([last_30, pd.Series(forecast.values, index=future_dates)])
    all_dates = all_values.index

    plt.figure(figsize=(12, 6))
    plt.plot(last_30.index, last_30.values, label='Observé (30 derniers jours)', color='blue', marker='o')
    plt.plot(future_dates, forecast.values, label='Prévision (10 jours)', color='red', marker='o')
    plt.fill_between(future_dates, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='red', alpha=0.2, label='IC 95%')
    plt.title("S&P 500 - 30 dernières valeurs et 10 prévisions SARIMAX")
    plt.xlabel("Date")
    plt.ylabel("S&P 500")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# ...existing code...
    
    # 6. Résumé du modèle
    print("\n6. Résumé du modèle:")
    print(forecaster.summary())

    # 7. AFFICHER LES PRÉDICTIONS FUTURES
    print("\n7. PRÉDICTIONS FUTURES")
    print("=" * 60)

    # Récupérer les prédictions
    forecast, conf_int = forecaster.forecast_with_exog(n_periods=10)

    # Créer les dates futures
    last_date = forecaster.data.index[-1]
    future_dates = pd.bdate_range(start=last_date + pd.Timedelta(days=1), periods=10)

    # Afficher les prédictions
    print(f"\nDernière valeur observée: {forecaster.data['cible'].iloc[-1]:.2f}")
    print(f"Date: {last_date.strftime('%Y-%m-%d')}\n")

    print("Prédictions pour les 10 prochains jours:")
    print("-" * 60)
    for i in range(len(forecast)):
        date = future_dates[i]
        price = forecast.iloc[i]
        lower = conf_int.iloc[i, 0]
        upper = conf_int.iloc[i, 1]
        
        print(f"{date.strftime('%Y-%m-%d')} | Prix: {price:7.2f} | IC 95%: [{lower:7.2f}, {upper:7.2f}]")

    # Variation attendue J+1
    change = forecast.iloc[0] - forecaster.data['cible'].iloc[-1]
    change_pct = (change / forecaster.data['cible'].iloc[-1]) * 100

    print("\n" + "=" * 60)
    print(f"💡 PRÉDICTION POUR DEMAIN:")
    print(f"   Prix prédit: {forecast.iloc[0]:.2f}")
    print(f"   Variation:   {change:+.2f} ({change_pct:+.2f}%)")
    print("=" * 60)


1. Création des variables techniques...
 1. close
 2. volume
 3. high
 4. low
 5. pe
 6. num_daily_adv_minus_decl
 7. mov_avg_20d
 8. best_eps
 9. dvd_sh_last
10. rsi_3d
11. rsi_9d
12. rsi_14d
13. rsi_30d
14. mov_avg_10d
15. mov_avg_30d
16. mov_avg_50d
17. pb
18. pib_pct
19. pib
20. vix
21. i
22. i_future
23. inflation
24. gold
25. gold_pct
26. gold_volume
27. brent
28. brent_pct
29. brent_volume
30. vol
31. score
32. momentum_10d
33. macd
34. macd_signal
35. bb_upper
36. bb_lower
37. mov_avg_100d
38. mov_avg_200d
39. mov_avg_10_50_diff
40. mov_avg_20_50_diff
41. close_minus_10d
42. close_minus_20d
43. close_minus_30d
44. close_minus_50d
45. close_minus_100d
46. close_minus_200d
47. DXY
48. EIA/GDPQXUS/USA
49. CroissM
50. volatility_5d
51. volatility_20d
52. ma_signal_short
53. ma_signal_long
54. volume_ma_20
55. volume_ratio

Corrélations avec les rendements:
   ma_signal_short: 0.297
   ma_signal_long: 0.182
   gold_pct: 0.036
   inflation: 0.027
Ajustement SARIMAX(1, 1, 1)x(3, 0, 1,

KeyboardInterrupt: 